In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df_sub = pd.read_csv('gender_submission.csv')

df_train 데이터

In [3]:
Name_list = set(['Dr', 'Master', 'Miss', 'Mr', 'Mrs', 'Ms'])
for i in range(len(df_train)):
    start = df_train.loc[i, 'Name'].index(',')
    end = df_train.loc[i, 'Name'].index('.')
    Name = df_train.loc[i, 'Name'][start+1:end].strip()

    if Name in Name_list: df_train.loc[i, 'Name'] = Name
    else: df_train.loc[i, 'Name'] = 'No'

female_mean_age = df_train['Age'][df_train['Sex'] == 'female'].mean()
male_mean_age = df_train['Age'][df_train['Sex'] == 'male'].mean()

df_train.loc[df_train['Sex'] == 'female', 'Age'] = df_train.loc[df_train['Sex'] == 'female', 'Age'].fillna(female_mean_age)
df_train.loc[df_train['Sex'] == 'male', 'Age'] = df_train.loc[df_train['Sex'] == 'male', 'Age'].fillna(male_mean_age)

df_train['Embarked'].fillna(df_train['Embarked'].mode()[0], inplace=True)
df_train.drop(['Ticket', 'Cabin'], axis=1, inplace=True)

df_train = pd.get_dummies(df_train, columns=['Pclass', 'Name', 'Sex', 'Embarked'], drop_first=True)

df_test 데이터

In [4]:
Name_list = set(['Dr', 'Master', 'Miss', 'Mr', 'Mrs', 'Ms'])
for i in range(len(df_test)):
    start = df_test.loc[i, 'Name'].index(',')
    end = df_test.loc[i, 'Name'].index('.')
    Name = df_test.loc[i, 'Name'][start+1:end].strip()

    if Name in Name_list: df_test.loc[i, 'Name'] = Name
    else: df_test.loc[i, 'Name'] = 'No'

female_mean_age = df_test['Age'][df_test['Sex'] == 'female'].mean()
male_mean_age = df_test['Age'][df_test['Sex'] == 'male'].mean()

df_test.loc[df_test['Sex'] == 'female', 'Age'] = df_test.loc[df_test['Sex'] == 'female', 'Age'].fillna(female_mean_age)
df_test.loc[df_test['Sex'] == 'male', 'Age'] = df_test.loc[df_test['Sex'] == 'male', 'Age'].fillna(male_mean_age)

df_test['Embarked'].fillna(df_test['Embarked'].mode()[0], inplace=True)
df_test.drop(['Ticket', 'Cabin'], axis=1, inplace=True)
df_test = pd.get_dummies(df_test, columns=['Pclass', 'Name', 'Sex', 'Embarked'], drop_first=True)

In [5]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

device = torch.device("mps:0" if torch.backends.mps.is_available() else "cpu")
device

device(type='mps', index=0)

In [6]:
x_train = df_train.iloc[:, 2:].values.astype('float32')
y_train = df_train.loc[:, 'Survived'].values.astype('long')

x_train = torch.tensor(x_train)
y_train = torch.tensor(y_train)

In [7]:
class Dataset(Dataset):
    def __init__(self, x_data, y_data):
        self.x_data = x_data
        self.y_data = y_data

    def __len__(self):
        return len(self.x_data)

    def __getitem__(self, idx):
        return self.x_data[idx], self.y_data[idx]

train_dataset = Dataset(x_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

In [8]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(15, 512)
        self.fc2 = nn.Linear(512, 512)
        self.fc3 = nn.Linear(512, 64)
        self.fc4 = nn.Linear(64, 2)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)
        self.sigmoid = nn.Sigmoid()
    def forward(self, x):
        x = self.fc1(x)
        x = self.dropout(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.dropout(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = self.dropout(x)
        x = self.relu(x)
        x = self.fc4(x)
        x = self.sigmoid(x)
        return x

model = Net().to(device)
model

Net(
  (fc1): Linear(in_features=15, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=2, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.2, inplace=False)
  (sigmoid): Sigmoid()
)

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [10]:
batch_size = 64
epochs = 500

train_loss_min = np.Inf
model.train()
for epoch in range(epochs):
    train_loss = 0
    num_correct = 0
    for idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * data.size(0)
        predict = torch.max(output, 1)[1]
        num_correct += (predict == target).sum().item()

    train_loss = train_loss / len(train_loader.dataset)

    if train_loss <= train_loss_min:
        print('Validation loss decreased ({:.6f} -> {:.6f}). Saving the model..'
              .format(train_loss_min, train_loss))
        torch.save(model.state_dict(), 'model.pt')
        train_loss_min = train_loss
    if (epoch + 1) % 100 == 0:
        print('Epoch: {} \tTraining Loss: {:.6f} \tTraining Accuracy: {:.2f}%'
              .format(epoch + 1, train_loss, num_correct / len(train_loader.dataset) * 100))

print('Training ended!')

Validation loss decreased (inf -> 0.624711). Saving the model..
Validation loss decreased (0.624711 -> 0.613815). Saving the model..
Validation loss decreased (0.613815 -> 0.601794). Saving the model..
Validation loss decreased (0.601794 -> 0.595499). Saving the model..
Validation loss decreased (0.595499 -> 0.593767). Saving the model..
Validation loss decreased (0.593767 -> 0.584147). Saving the model..
Validation loss decreased (0.584147 -> 0.570252). Saving the model..
Validation loss decreased (0.570252 -> 0.544651). Saving the model..
Validation loss decreased (0.544651 -> 0.542816). Saving the model..
Validation loss decreased (0.542816 -> 0.521987). Saving the model..
Validation loss decreased (0.521987 -> 0.517699). Saving the model..
Validation loss decreased (0.517699 -> 0.511349). Saving the model..
Validation loss decreased (0.511349 -> 0.510011). Saving the model..
Validation loss decreased (0.510011 -> 0.507305). Saving the model..
Validation loss decreased (0.507305 -> 

In [11]:
x_test = df_test.iloc[:, 1:].values.astype('float32')
x_test = torch.tensor(x_test).to(device)

model.eval()
with torch.no_grad():
    result = model(x_test)
values, labels = torch.max(result, 1)
survived = labels.cpu().data.numpy()

In [12]:
submission = pd.DataFrame({'PassengerId': df_sub['PassengerId'], 'Survived': survived})
submission.to_csv('submission.csv', index=False)